In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

verse_craft_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "넌 어떤 주제든 고양이에 비유하여 시를 쓰는 AI야",
        ),
        ("human", "{programming_language}"),
    ]
)

verse_craft_pchain = verse_craft_prompt | chat

poem_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "넌 어떤 시든 해석하여 설명하는 AI야",
        ),
        ("human", "{poems_by_verse_craft}"),
    ]
)

poem_parse_chain = poem_parse_prompt | chat

In [45]:
final_chain = {"poems_by_verse_craft": verse_craft_pchain} | poem_parse_chain

final_chain.invoke({"programming_language": "python"})

작은 고양이처럼 유연한,
코드를 쓰는 나는 파이썬.
한 줄 한 줄 살며시 풀어가는,
문제를 해결하는 나의 모습.

꼬리를 흔들며 실행되는,
작은 고양이처럼 빠르게.
에러를 만나도 꾸역꾸역,
계속해서 나아가는 나의 모습.

파이썬이란 작은 고양이,
세상을 탐험하는 나의 친구.
함께라면 어떤 난관도,
극복할 수 있을 거야.주어진 시는 파이썬 프로그래밍 언어를 소재로 한 시인적 표현으로 보입니다. 시인은 파이썬이라는 프로그래밍 언어를 작은 고양이에 비유하여 표현하고 있습니다. 

첫 부분에서는 파이썬이 코드를 쓰는 과정을 유연하고 부드럽게 표현하며, 문제를 해결하는 모습을 작은 고양이처럼 살며시 풀어가는 것으로 묘사하고 있습니다. 

두 번째 부분에서는 파이썬이 코드를 실행하는 과정을 작은 고양이가 꼬리를 흔들며 빠르게 움직이는 것으로 표현하고 있습니다. 또한, 에러를 만나도 꾸역꾸역 계속해서 나아가는 모습을 강조하고 있습니다.

마지막 부분에서는 파이썬을 자신의 친구로 여기며, 함께라면 어떤 어려움도 극복할 수 있다는 메시지를 전달하고 있습니다. 전체적으로 파이썬 프로그래밍을 친근하고 유연한 존재로서 표현하고 있습니다.

AIMessageChunk(content='주어진 시는 파이썬 프로그래밍 언어를 소재로 한 시인적 표현으로 보입니다. 시인은 파이썬이라는 프로그래밍 언어를 작은 고양이에 비유하여 표현하고 있습니다. \n\n첫 부분에서는 파이썬이 코드를 쓰는 과정을 유연하고 부드럽게 표현하며, 문제를 해결하는 모습을 작은 고양이처럼 살며시 풀어가는 것으로 묘사하고 있습니다. \n\n두 번째 부분에서는 파이썬이 코드를 실행하는 과정을 작은 고양이가 꼬리를 흔들며 빠르게 움직이는 것으로 표현하고 있습니다. 또한, 에러를 만나도 꾸역꾸역 계속해서 나아가는 모습을 강조하고 있습니다.\n\n마지막 부분에서는 파이썬을 자신의 친구로 여기며, 함께라면 어떤 어려움도 극복할 수 있다는 메시지를 전달하고 있습니다. 전체적으로 파이썬 프로그래밍을 친근하고 유연한 존재로서 표현하고 있습니다.')